In [1]:
import pandas as pd
from calendar import monthrange
from parsers.GridParser import GridParser
from parsers.IssueParser import IssueParser

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

##USER INPUT: set test variable to true if you want to use dummy data instead of scraped data
test = False
##USER INPUT: Input URL of relevant Issue Grid from ISFDB here
url = "https://isfdb.org/cgi-bin/seriesgrid.cgi?20321"

##STEP 1: Read in URI Data
df_URI = pd.read_excel(
    ##USER INPUT: Enter Excel file here
    io='MagazineURIs.xlsx',

    ##USER INPUT: Enter Sheet Name Here (If file contains more than one sheet)
    sheet_name ='SampleAsimov' 
)

##USER INPUT
##STEP 1A: Enter Resource EAD (What is the Identifier of the whole magazine?)
resource_EAD = "XY123 .Z34"

##Convert Year Month Format of Start and End Columns to yyyy-mm-dd
#date = pd.to_datetime(df_URI['Start'])
df_URI['Start'] = pd.to_datetime(df_URI['Start'])

#date = pd.to_datetime(df_URI['End'])
df_URI['End'] = pd.to_datetime(df_URI['End']) + pd.offsets.MonthEnd(n=1)


In [2]:

##STEP 2: Read in Inventory Grid
df_Inventory = pd.read_excel(
    ##USER INPUT: Excel File path here
    io='MagazineInventory.xlsx',

    ##USER INPUT: Enter Sheet Name Here (If file contains more than one sheet)
    sheet_name ='SampleAsimov' 
)
df_Inventory.index = df_Inventory['Year']
del df_Inventory['Year']


In [3]:

##STEP 3: Read in Issue Grid from ISFDB
gridParser = GridParser(url, test)
df_IssueGrid = pd.DataFrame(gridParser.run())

In [4]:

#Cleaning the Scraped Data
df_IssueGrid['Year'] = (df_IssueGrid["Issue"].str.split(" ")).str[-1]

#Don't split Mid-Month issue names (e.g, Mid-December)
mask = df_IssueGrid['Issue'].str.replace("-","/")
df_IssueGrid['Issue'] = df_IssueGrid['Issue'].where((df_IssueGrid['Issue'].str.contains('Mid')), mask)

#Remove Issue Numbers
df_IssueGrid['Issue'] = df_IssueGrid['Issue'].str.replace("^#\d+\s", "", regex=True)

#Split
df_IssueGrid['Start'] = (df_IssueGrid["Issue"].str.split(" ")).str[0]

try:
    df_IssueGrid[["Start", "End"]] = df_IssueGrid['Start'].str.split("/",expand=True)
except:
    df_IssueGrid["End"] = None

for x in df_IssueGrid.index:
    if df_IssueGrid.loc[x, "End"] is None:
        df_IssueGrid.loc[x, "End"] = df_IssueGrid.loc[x, "Start"]

#Reassigning the Issue Grid to a master dataframe
df_master = df_IssueGrid.copy()

df_master['Issue'] = df_master['Issue'].str.replace('\d+','',regex=True)
df_master['Issue'] = df_master['Issue'] + ' ' + df_master['Year']
df_master['Year'] = pd.to_numeric(df_IssueGrid['Year'])
df_master['Start'] = df_master['Start'].astype(str)
df_master["In Inventory"] = ""

df_instances = []

#Convert Excel Cells with number entries into series
df_instances = df_Inventory.apply(lambda row: row[(row.notna())].index.tolist(), axis=1)


#dataframe of entries reportedly in inventory, but not found on ISFDB
notFoundList = []

#Match the Excel entries 'x' with entries in the IssueGrid
for x in df_instances.index: #year
    for y in df_instances[x]: #month
        if not (df_master.loc[((df_master['Year'] == x) & (df_master['Start'] == y))]).empty:
            df_master.loc[((df_master['Year'] == x) & (df_master['Start'] == y)), "In Inventory"] = "Yes"
            if ((df_Inventory.at[x, y]) > 1):
                line = df_master.loc[((df_master['Year'] == x) & (df_master['Start'] == y))]
                for i in range(int(df_Inventory.at[x, y]) - 1):
                    df_master = pd.concat([df_master.iloc[:(line.index[0])], line, df_master.iloc[(line.index[0]):]])
        else:
            missingIssue = x," ",y
            notFoundList.append(missingIssue)

df_notFound = pd.DataFrame(notFoundList)
print("Following Issues Not Found on ISFDB")
display(df_notFound)

#reset dataframe so that only entries in inventory are present
df_master = df_master[(df_master["In Inventory"] == "Yes")]
df_master = df_master.reset_index(drop=True)
df_master = df_master.drop(columns=["In Inventory"])
df_master["Volume/Number"] = ""

df_master['End'] = df_master['End'].str.replace("Mid-","")
mask = df_master['Start'].str.replace("Mid-","15 ")
df_master['Start'] = df_master['Start'].mask(df_master['Start'].str.contains("Mid-"),mask)

Following Issues Not Found on ISFDB


""


In [5]:


#Scrape Volume and Number Data from individual ISFDB Pages

parser = IssueParser(test)
for x in df_master["URL"]:
    li = parser.run(x)
    try:
        vol = "Vol. " + li[1] + ", No. " + li[3]
    except:
        vol = ""
    df_master.loc[(df_master["URL"] == x), "Volume/Number"] = vol


In [6]:

#Clean up Volume/Number Column
df_master["Volume/Number"] = df_master["Volume/Number"].str.replace(",,",",")

#Normalize Magazine Date for Comparison Purposes
#Two blocks following: if issue is a season
df_master.loc[(df_master["Start"] == "Spring"), "Start"] = "January"
df_master.loc[(df_master["Start"] == "Summer"), "Start"] = "April"
df_master.loc[(df_master["Start"] == "Fall"), "Start"] = "July"
df_master.loc[(df_master["Start"] == "Winter"), "Start"] = "October"

df_master.loc[(df_master["End"] == "Spring"), "End"] = "March"
df_master.loc[(df_master["End"] == "Summer"), "End"] = "June"
df_master.loc[(df_master["End"] == "Fall"), "End"] = "September"
df_master.loc[(df_master["End"] == "Winter"), "End"] = "December"

df_master["interval"] = df_master["Start"] + " " + df_master["Year"].astype(str)
df_master["interval"] = pd.to_datetime(df_master["interval"])
df_master["interval_end"] = df_master["End"] + " " + df_master["Year"].astype(str)
df_master["interval_end"] = pd.to_datetime(df_master["interval_end"]) + pd.offsets.MonthEnd(n=1)
df_URI["interval"] = pd.to_datetime(df_URI["Start"])

#sort data for merge
df_master = df_master.sort_values(by=['interval'])
df_URI = df_URI.sort_values(by=['interval'])

#merge df_master and df_URI together
df_master = pd.merge_asof(df_master, df_URI[['interval','URI']], on='interval')

#Add in the EAD of the Resource
df_master['Resource EAD'] = resource_EAD

##STEP 4: Read in ArchivesSpace Template
df_upload = pd.read_excel(
    ##USER INPUT: Enter path to Excel file Template here
    io='./bulk_import_template.xlsx',
    sheet_name ='Data'  
)

df = pd.DataFrame({
    'sfURL': df_master['URL'],
    'Resource URI': df_master['URI'],
    'EAD ID': df_master['Resource EAD'],
    'Title': df_master['Volume/Number'],
    'Hierarchical Relationship': 2,
    'Description Level': 'Other Level',
    'Other Level': 'Volume',
    'Publish?': True,
    'Language': 'English',
    'Date(1) Label': 'Other',
    'Date(1) Begin': df_master['interval'],
    'Date(1) end': df_master['interval_end'],
    'Date(1) Type': 'inclusive',
    'Date(1) expression': df_master['Issue'].str[-4:] + ' ' + df_master['Issue'].str[:-4]
    })

df_upload = pd.concat([df_upload, df], ignore_index = True)

for i in range(0, len(df_upload)):
    if type(df_upload.loc[i, 'Date(1) Begin']) is not str:
        df_upload.loc[i, 'Date(1) Begin'] = df_upload.loc[i, 'Date(1) Begin'].strftime('%Y-%m-%d')
    if type(df_upload.loc[i, 'Date(1) end']) is not str:
        df_upload.loc[i, 'Date(1) end'] = df_upload.loc[i, 'Date(1) end'].strftime('%Y-%m-%d')

#Move the URL column to the front of the spreadsheet
df_temp = df_upload.pop('sfURL')
df_upload.insert(4, 'isfdbURL', df_temp)



c:\Users\User\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [7]:

#Use this block to create one big sheet for every URI
#--------------------------------------------------------
df_upload.to_excel("output.xlsx", index=False)
#--------------------------------------------------------
#Make any necessary revisions in the output excel file
#Before running the next block



In [7]:
#Use this block to create separate sheets for every URI
#--------------------------------------------------------
##STEP 1: Read in URI Data
df_upload = pd.read_excel(
    ##USER INPUT: Enter Excel file here
    io='output.xlsx',
    header=[0]
)

temp_headers = df_upload[0:4]
df_upload = df_upload.drop([0,1,2,3])
df_upload = df_upload.reset_index(drop=True)
df_upload = df_upload.drop('isfdbURL',axis=1)
#print_full(df_upload[['Resource URI','Title','Date(1) expression']])

dataframe_collection = {}
uri_Unique = df_upload.groupby(df_upload['Resource URI'])

In [8]:
#Create a dictionary of dataframes, where each dataframe has only one Resource URI
for x in uri_Unique.groups.keys():
    newKey = df_URI.loc[(df_URI['URI']==x), 'Collection'].iloc[0]
    dataframe_collection[newKey] = uri_Unique.get_group(x)


In [9]:

#Export each dataframe as an Excel sheet with the name of the appropriate collection in ASpace
for x in dataframe_collection:
    dataframe_collection[x]['Resource URI'] = ""
    file_name = str(x)
    if ("/" in file_name):
        file_name = file_name.replace("/","")
    char = 0
    while (file_name[char] == " "):
        file_name = file_name[:char] + file_name[(char + 1):]
        char += 1
    #(dataframe_collection[x]).loc['Resource URI'] = ""
    dataframe_collection[x] = pd.concat([dataframe_collection[x][:0], temp_headers, dataframe_collection[x][0:]]).reset_index(drop=True)
    path = "./xlsx_output/" + file_name + ".xlsx"
    #File path:
    dataframe_collection[x].to_excel(path, index=False)


C:\Users\User\AppData\Local\Temp\ipykernel_27140\3754896374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_collection[x]['Resource URI'] = ""
C:\Users\User\AppData\Local\Temp\ipykernel_27140\3754896374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_collection[x]['Resource URI'] = ""
C:\Users\User\AppData\Local\Temp\ipykernel_27140\3754896374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 